In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow.train as ttrain
import functools as ft
import numpy as np
data_path="/Users/ijeonghun/Documents/projects/dacon/웹광고/train.csv"

In [3]:
# 데이터 타입
df=pd.read_csv(data_path,nrows=10)
df_copy=df.copy()
df_copy=df_copy.dropna(axis=0)
dtype_dict=dict(df_copy.dtypes)

In [4]:
# str 타입 -> bytes 타입으로 변환
def trans_dytes(data:str):
    return data.encode("utf-8")

In [40]:
# 데이터 타입 매칭
def dtype_mapping(feature:str,dtypes:dict,data:str):
    '''변환을 위한 함수'''
    if dtype_dict[feature]==np.dtype("O"):
        return ttrain.Feature(bytes_list=ttrain.BytesList(value=[trans_dytes(data)]))
    elif dtype_dict[feature]==np.dtype("int64"):
        return ttrain.Feature(int64_list=ttrain.Int64List(value=[int(data) if data else 0]))
    else:
        return ttrain.Feature(float_list=ttrain.FloatList(value=[float(data) if data else 0.0]))
d_map=ft.partial(dtype_mapping,dtypes=dtype_dict)

In [4]:
with open(data_path,"r") as f:
    file=f.readlines()

In [5]:
def csv_reader(data:str):
    return list(map(lambda x:x.strip(),data.split(",")))

In [10]:
d_map(feature="ID",data="이정훈")

value: "\354\235\264\354\240\225\355\233\210"

In [41]:
feature_names=csv_reader(file[0])
test_zip=zip(feature_names,csv_reader(file[1]))
test_feature={
                    x:d_map(feature=x,data=y) for x,y in test_zip
                }

In [44]:
with tf.io.TFRecordWriter("output.tfrecord") as writer:
    feature_names=csv_reader(file[0])
    for data in file[1:]:
        temp_zip=zip(feature_names,csv_reader(data))
        example=ttrain.Example(
            features=ttrain.Features(
                feature={
                    x:d_map(feature=x,data=y) for x,y in temp_zip
                }
            )
        )
        writer.write(example.SerializeToString())

KeyboardInterrupt: 